In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import re

ener_train = pd.read_csv('training.csv')
ener_train

ener_test = pd.read_csv('testing.csv')
ener_test

ene = pd.concat([ener_train, ener_test],axis=0)
ene['date'] = pd.to_datetime(ene['date'])
ene.sort_values(by='date', inplace=True)
ene.iloc[:,:-3]

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,...,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,...,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,...,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
0,2016-01-11 17:30:00,50,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,...,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
3,2016-01-11 17:40:00,60,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,...,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4930,2016-05-27 17:20:00,100,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,...,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812,43.096812
14800,2016-05-27 17:30:00,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,...,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940,49.282940
4931,2016-05-27 17:40:00,270,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,...,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117,29.199117
14801,2016-05-27 17:50:00,420,10,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,...,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333,6.322784,6.322784


In [ ]:
# check morning peak  or evening peak
ett = ene.iloc[:,:-3]
ett['date'] = pd.to_datetime(ett['date'])
mask = (ett['date'].dt.time >= pd.Timestamp('07:00:00').time()) & (ett['date'].dt.time <= pd.Timestamp('10:00:00').time())
mask = (ett['date'].dt.time >= pd.Timestamp('07:00:00').time()) & (ett['date'].dt.time <= pd.Timestamp('10:00:00').time())
filtered_data = ett.loc[mask]


# filtered_data.to_csv('morning_peak_energy.csv', index=False)

filtered_data

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
62,2016-01-12 07:00:00,40,10,19.823333,46.863333,19.200000,45.363333,20.200000,45.290000,19.290000,...,17.20,50.000000,5.1,741.100000,93.000000,5.000000,29.000000,4.100000,37.854357,37.854357
63,2016-01-12 07:10:00,50,10,19.856667,46.930000,19.200000,45.400000,20.200000,45.290000,19.290000,...,17.20,50.090000,5.1,741.216667,92.666667,5.000000,30.833333,4.050000,10.134613,10.134613
22,2016-01-12 07:20:00,60,20,19.790000,46.790000,19.200000,45.326667,20.166667,45.290000,19.290000,...,17.20,49.963333,5.1,741.333333,92.333333,5.000000,32.666667,4.000000,35.556061,35.556061
64,2016-01-12 07:30:00,60,20,19.790000,46.895000,19.175000,45.195000,20.100000,45.290000,19.260000,...,17.20,49.723333,5.1,741.450000,92.000000,5.000000,34.500000,3.950000,6.084596,6.084596
65,2016-01-12 07:40:00,70,20,19.790000,47.426667,19.100000,45.030000,20.100000,45.290000,19.200000,...,17.20,49.463333,5.1,741.566667,91.666667,5.000000,36.333333,3.900000,12.274426,12.274426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4924,2016-05-27 09:20:00,70,0,23.600000,47.200000,24.185000,44.243750,26.100000,40.466667,24.390000,...,22.89,46.500000,15.5,756.133333,80.666667,2.333333,61.000000,12.133333,38.128046,38.128046
14758,2016-05-27 09:30:00,50,0,23.600000,47.200000,24.452857,43.750000,26.100000,40.466667,24.390000,...,22.89,46.500000,15.7,756.150000,80.000000,2.000000,61.000000,12.200000,31.845927,31.845927
14759,2016-05-27 09:40:00,580,0,23.633333,47.226667,24.814000,43.472000,26.166667,40.500000,24.500000,...,22.89,46.433333,15.9,756.166667,79.333333,1.666667,61.000000,12.266667,27.740214,27.740214
14760,2016-05-27 09:50:00,660,0,23.760000,47.693333,25.497143,42.595714,26.033333,40.693333,24.500000,...,22.89,46.290000,16.1,756.183333,78.666667,1.333333,61.000000,12.333333,4.105907,4.105907


In [3]:
# check morning peak  or evening peak
ett = ene.iloc[:,:-3]
ett['date'] = pd.to_datetime(ett['date'])
mask = (ett['date'].dt.time >= pd.Timestamp('17:00:00').time()) & (ett['date'].dt.time <= pd.Timestamp('20:00:00').time())
mask = (ett['date'].dt.time >= pd.Timestamp('17:00:00').time()) & (ett['date'].dt.time <= pd.Timestamp('20:00:00').time())
filtered_data = ett.loc[mask]


# filtered_data.to_csv('evening_peak_energy.csv', index=False)

filtered_data

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,...,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,...,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,...,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
0,2016-01-11 17:30:00,50,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,...,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
3,2016-01-11 17:40:00,60,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,...,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4930,2016-05-27 17:20:00,100,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,...,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812,43.096812
14800,2016-05-27 17:30:00,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,...,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940,49.282940
4931,2016-05-27 17:40:00,270,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,...,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117,29.199117
14801,2016-05-27 17:50:00,420,10,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,...,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333,6.322784,6.322784


In [38]:
day_ett = pd.read_csv('day_etth.csv')
day_ett

,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 06:00:00,7.167,2.947,2.132,0.782,5.026,1.858,22.792000
1,2016-07-01 07:00:00,7.435,3.282,2.310,1.031,5.087,2.224,23.143999
2,2016-07-01 08:00:00,5.559,3.014,2.452,1.173,2.955,1.432,21.667000
3,2016-07-01 09:00:00,4.555,2.545,1.919,0.817,2.680,1.371,17.445999
4,2016-07-01 10:00:00,4.957,2.545,1.990,0.853,2.955,1.492,19.979000
...,...,...,...,...,...,...,...,...
8707,2018-06-26 13:00:00,-6.229,3.483,-9.666,1.528,3.137,1.310,9.426000
8708,2018-06-26 14:00:00,-1.273,3.617,-4.904,2.132,3.320,1.432,10.200000
8709,2018-06-26 15:00:00,-1.674,3.550,-5.615,2.132,3.472,1.523,10.904000
8710,2018-06-26 16:00:00,-5.492,4.287,-9.132,2.274,3.533,1.675,11.044000
